# Benchmarking pour un dossier de prédictions


@Author Colin de Seroux


## <span style="color:lightblue">Installation des dépendances</span>


In [ ]:
%pip install -q -r requirements.txt

## <span style="color:lightblue">Importation des dépendances</span>


In [17]:
import pandas as pd
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

from utils import extract_annotations_from_file, extract_predictions_from_folder, extract_metrics, get_coco_predictions, evaluate_metrics, extract_inference_time, plot_inference_time, plot_inference_time_mAP_50_95

## <span style="color:lightblue">Code principale</span>


### <span style="color:lightgreen">Environnement</span>


In [76]:
ANNOTATIONS_FILE_PATH = "../data/dataset4/test/result.json"
PREDICTIONS_DIR = "../results/predictions"
DATASET_NAME = "DATASET 4"
COMPARED_TYPE = f"{DATASET_NAME} YOLO"
INFERENCE_TIME_GOAL = 200

### <span style="color:lightgreen">Récupération des données</span>


In [ ]:
y_true, class_names = extract_annotations_from_file(ANNOTATIONS_FILE_PATH)
y_preds, y_scores = extract_predictions_from_folder(ANNOTATIONS_FILE_PATH, PREDICTIONS_DIR)
coco_annotations = COCO(ANNOTATIONS_FILE_PATH)
coco_predictions = get_coco_predictions(coco_annotations, PREDICTIONS_DIR)

### <span style="color:lightgreen">mAP</span>


Initialiser l'évaluation


In [78]:
coco_evals = []

for coco_prediction in coco_predictions:
    coco_eval = COCOeval(coco_annotations, coco_prediction, iouType="bbox")
    coco_evals.append(coco_eval)

Calculer les AP et AR


In [ ]:
for coco_eval in coco_evals:
	coco_eval.evaluate()
	coco_eval.accumulate()
	coco_eval.summarize()

### <span style="color:lightgreen">Récupération des metrics</span>


In [80]:
metrics = []

for coco_eval in coco_evals:
    precision, recall, scores, iou_lookup = evaluate_metrics(coco_eval)
    current_metrics = extract_metrics(precision, recall)
    metrics.append(current_metrics)

### <span style="color:lightgreen">Traitement des données</span>


In [81]:
prediction_files, inference_times, average_inference_time = extract_inference_time(PREDICTIONS_DIR)

### <span style="color:lightgreen">Comparaison uniquement sur le temps</span>


In [ ]:
plot_inference_time(prediction_files, inference_times, average_inference_time, INFERENCE_TIME_GOAL, COMPARED_TYPE)

### <span style="color:lightgreen">Nuage de points pour une comparaison simplifiée</span>


In [ ]:
plot_inference_time_mAP_50_95(prediction_files, inference_times, average_inference_time, metrics, INFERENCE_TIME_GOAL, DATASET_NAME, COMPARED_TYPE)

### <span style="color:lightgreen">Comparaison sous forme de tableau avec le F1-score</span>


In [ ]:
data = []

for metric, inference_time, prediction_file in zip(metrics, inference_times, prediction_files):
    data.append({
        "Nom du modèle": prediction_file,
        "mAP 50-95": metric["mAP_50_95"],
        "F1-score": metric["f1_score_50_95"],
        "Temps d'inférence moyen": inference_time
    })

df_metrics = pd.DataFrame(data)

df_metrics.style.format({
    "mAP 50-95": "{:.3}".format,
    "F1-score": "{:.3}".format,
    "Temps d'inférence moyen": "{:.4}ms".format
})

df_metrics.to_csv(f"../results/benchmarking/{COMPARED_TYPE}_metrics.csv", index=False)

df_metrics = df_metrics.sort_values(by="Temps d'inférence moyen")
df_metrics

## <span style="color:lightblue">Sources</span>


- https://gist.github.com/shivamsnaik/c5c5e99c00819d2167317b1e56871187
- https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.f1_score.html
- https://kobia.fr/classification-metrics-f1-score
